In [170]:
with open('data/songs/爱得太迟.txt') as file:
    content = file.read().splitlines()
    
file.close()

clean_l = [x for x in content if x != '' and 'https' not in x]

In [171]:
# Split chinese and english
cn_l, en_l = clean_l[::2], clean_l[1::2]

In [172]:
# Split tones
import string

dict_table = str.maketrans('', '', string.digits)
clean_en_l = [x.translate(dict_table) for x in en_l]

In [ ]:
from termcolor import colored

red = "\033[31m"
green = "\033[32m"
reset = "\033[39m"

word_book = {}
word_info = []

for s in range(2):
    print(cn_l[s])
    
    i = input()

    correct_cn_w = list(cn_l[s].replace(' ', ''))
    correct_words = clean_en_l[s].split(' ')
    colored_correct_words = correct_words.copy()
    check_words = i.split(' ')

    for w in range(len(check_words)):

        if (check_words[w] != correct_words[w]) and ('/' not in correct_words[w] or check_words[w] not in correct_words[w].split('/')):
            check_words[w] = red + check_words[w] + reset
            colored_correct_words[w] = green + correct_words[w] + reset

            if correct_cn_w[w] in word_book:
                word_book[correct_cn_w[w]][1] += 1
            else:
                word_book[correct_cn_w[w]] = [correct_words[w], 1]

    output_lyrics = ' '.join(check_words)
    correct_lyrics = ' '.join(colored_correct_words)    

    print(correct_lyrics)
    print(output_lyrics)

In [ ]:
# output json word book to file
import json

with open('data/new words.json', 'w') as f:
    json.dump(word_book, f, indent=4, ensure_ascii=False)

f.close()

In [ ]:
import pandas as pd

# load json word book from file
f = open('data/new words.json', 'r')

js_wb = json.loads(f.read())
js_wb

In [ ]:
# output as dataframe
data = [[key] + values for key, values in js_wb.items()]
df = pd.DataFrame(data, columns=['Word', 'Pronunciation', 'Counts'])
df

In [ ]:
# check existing
import os

if not os.path.isfile('data/new words.txt'):
    open('data/new words.txt', 'a').close()
else:
    print('New words book is existing.')

In [ ]:
# get list of exisitng songs
import glob

all_f = glob.glob('data/songs/*.txt')
all_songs = [x[11:-4] for x in all_f]
all_songs

In [ ]:
title_l1 = "Let's start to learn Cantonese!!!"

game_start = f"+{'-'*(20+len(title_l1))}+\n|{' '*(20+len(title_l1))}|\n|{' '*10}{title_l1}{' '*10}|\n|{' '*(20+len(title_l1))}|\n+{'-'*(20+len(title_l1))}+\n"
print(game_start)

In [ ]:
# remove redundent spaces
import re
s = '   this is a text for     finding   multiple spaces'
re.sub('\s+', ' ', s).lstrip()


In [ ]:
import learnCantonese as lc
W_DIR = 'data/new words.json'

new_w = dict(lc.read_new_words(W_DIR).items())
new_w_list = [f'{i[0]}: {i[1][0]}' for i in new_w.items()]

new_w[new_w_list[0].split(':')[0]].append(1)

new_w

In [ ]:
    return '\n\n'.join(['\n'.join(x) for x in zip(lyrics[::2], lyrics[1::2])])


In [ ]:
# Tidy up and align lyrics
import learnCantonese as lc
import re

# Split lyrics for better reading
lyric_dir = 'data/songs/南瓜车.txt'
clean_l = list(filter(None, lc.show_lyrics(lyric_dir).split('\n')))
cn_l, en_l = clean_l[::2], clean_l[1::2]
en_l_format = [len(y) for y in (re.sub('\s+', ' ', x).lstrip().split(' ') for x in en_l)]


In [ ]:
# Tidy up and align lyrics
import learnCantonese as lc
import re

# Split lyrics for better reading
lyric_dir = 'data/songs/南瓜车.txt'
clean_l = list(filter(None, lc.show_lyrics(lyric_dir).split('\n')))
cn_l, en_l = clean_l[::2], clean_l[1::2]

In [ ]:
import pandas as pd
t_en_l = [re.sub('\s+', ' ', x).lstrip().split(' ') for x in en_l]
l_s = [len(x) for x in en_l]
l = [[len(x) for x in y] for y in t_en_l]
t_cn_l = [list(re.sub('\s+', '', x)) for x in cn_l]

In [ ]:
test_c, test_e, test_l = t_cn_l[2], [en_l[2]], l[2]
t_o = ''
for i in range(len(test_c)):
    print(test_c[i], test_l[i], ':', test_c[i].center(test_l[i]))
    t_o += test_c[i].center(test_l[i])

print(t_o + '\n' + test_e[0])

In [ ]:

o_l = [] 
for i in range(len(t_cn_l)):
    o_s = ''.join(f'{t_cn_l[i][j].center(int(l_s[i]/(len(cn_l[i]))))}' for j in range(len(t_cn_l[i])))
    o_l.append(o_s)
print('\n\n'.join(['\n'.join(x) for x in zip(o_l, en_l)]))

In [ ]:
o_l = []
for i in range(len(t_cn_l)):
    o_s = ''.join(f'{t_cn_l[i][j].center(l[i][j]-1, chr(12288))}' for j in range(len(t_cn_l[i])))
    o_l.append(o_s)
print('\n\n'.join(['\n'.join(x) for x in zip(o_l, en_l)]))

In [ ]:
print(pd.concat([pd.DataFrame(t_cn_l), pd.DataFrame(t_en_l)]).sort_index())

#### Extract songs from webpage

In [ ]:
from bs4 import BeautifulSoup

data = '<li><a href="/zh-hans/singer/1">陈奕迅</a> <small>Eason Chan</small></li>'
soup = BeautifulSoup(data)
soup.find('a').contents[0]
soup.find('a')['href']

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_part(url, part):
    """ extract part from a url
    """
    link = url
    f = requests.get(link).content

    return BeautifulSoup(f, 'html.parser').find(part)

In [ ]:
url = "https://www.feitsui.com/singer_s/"
parent = get_part(url, 'body')

singers_id = []
songs_id = []

for ul in parent.find_all("ul", {'class': 'list-unstyled accessible'}):
    for li in ul.find_all('a'):
        singers_id.append(int(li['href'].split('/')[-1]))

# finding all song ids
for id in singers_id:
    url = f'https://www.feitsui.com/zh-hans/singer/{id}'
    singer = (get_part(url, 'head').find('title').contents[0].split(' ')[0])

    print(f'{singers_id.index(id)+1}/{len(singers_id)} Extract song list from {singer}')

    s_body = get_part(url, 'body')

    for s_ul in s_body.find_all("ul", {'class': 'list-unstyled accessible'}):
        for s_li in s_ul.find_all('a'):
            songs_id.append([int(s_li['href'].split('/')[-1]), singer, s_li.contents[0]])

res = pd.DataFrame(songs_id, columns=['song id', 'singer', 'song'])

# remove duplicate from multiple edition
res = res.drop_duplicates(subset='song id').drop_duplicates(subset=['singer', 'song'])
res.to_csv('data/songs info.csv')

In [9]:
# search song
song_df = pd.read_csv('data/songs info.csv').drop_duplicates(subset='song id').drop_duplicates(subset=['singer', 'song'])
select_s = '爱得太迟'

# display all results(different songs with the same name)
all_res = song_df[song_df['song'] == select_s]

# select target song
select_id = all_res.iloc[0, 1]
print(select_id)

# get lyric
lyric_f = open(f'data/songs/{select_s}.txt', 'w')
url = f'https://www.feitsui.com/zh-hans/lyrics/{select_id}'
parent = get_part(url, 'body')
for p in parent.find('article').find_all('p'):
    for l in p.contents:
        if '<' not in str(l):
            print(str(l))
            lyric_f.write(str(l).replace('翡翠粤语歌词', '').strip() + '\n')
lyric_f.close()

      Unnamed: 0  song id singer   song
736          736     3049    张国荣  但愿人长久
2065        2065     5943     王菲  但愿人长久
3843        3843     3050    陈百强  但愿人长久
5698        5698     6483    菊梓乔  但愿人长久
3049


In [4]:
import pandas as pd

song_df = pd.read_csv('data/songs info.csv').drop_duplicates(subset='song id').drop_duplicates(subset=['singer', 'song'])
select_s = '友谊万岁'
all_res = song_df[song_df['song'] == select_s]
all_res

,Unnamed: 0,song id,singer,song
514,514,966,杨千嬅,友谊万岁
3473,3473,968,苏永康,友谊万岁
5883,5883,967,汪明荃,友谊万岁
7159,7159,970,温拿乐队,友谊万岁


In [14]:
all_res[['singer', 'song']].values.tolist()

[['杨千嬅', '友谊万岁'], ['苏永康', '友谊万岁'], ['汪明荃', '友谊万岁'], ['温拿乐队', '友谊万岁']]

+-----------------------------------------------------+

         TKINTER EXAMPLES         

+-----------------------------------------------------+


### tkinter

#### Practise

In [ ]:
#!/usr/bin/env python3

import tkinter as tk
from tkinter.font import Font

class Pad(tk.Frame):

    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)

        self.toolbar = tk.Frame(self, bg="#eee")
        self.toolbar.pack(side="top", fill="x")

        self.bold_btn = tk.Button(self.toolbar, text="Bold", command=self.make_bold)
        self.bold_btn.pack(side="left")

        self.clear_btn = tk.Button(self.toolbar, text="Clear", command=self.clear)
        self.clear_btn.pack(side="left")

        # Creates a bold font
        self.bold_font = Font(family="Helvetica", size=14, weight="bold")

        self.text = tk.Text(self)
        self.text.insert("end", "Select part of text and then click 'Bold'...")
        self.text.focus()
        self.text.pack(fill="both", expand=True)

        # configuring a tag called BOLD
        self.text.tag_configure("BOLD", font=self.bold_font)

    def make_bold(self):
        # tk.TclError exception is raised if not text is selected
        try:
            self.text.tag_add("BOLD", "sel.first", "sel.last")        
        except tk.TclError:
            pass

    def clear(self):
        self.text.tag_remove("BOLD",  "1.0", 'end')


def demo():
    root = tk.Tk()
    Pad(root).pack(expand=1, fill="both")
    root.mainloop()


if __name__ == "__main__":
    demo()

In [ ]:
from tkinter import *

root = Tk()

txt = Text(root)
txt.pack()

s = ['white', 'white', 'red', 'red', 'white', 'white', 'white', 'white']

txt.tag_config('warning', foreground="red")

for i in s:
    if i == 'white':
        txt.insert('insert', f'{i} ')
    else:
        txt.insert('insert', i, 'warning')
        txt.insert('insert', ' ')

root.mainloop()

In [ ]:
from tkinter import *
from tkinter import font

root = Tk()

fontsize = 16

def increase_font():
    global fontsize
    fontsize += 2
    textfont.config(size=fontsize)

textfont = font.Font(family = "consolas" , size = fontsize)
# set the width and height of the Text widget to smallest values
# and let the layout manager to expand it
my_text = Text(root , width=1 , height=1 , font=textfont)
my_text.pack(fill="both", expand=1) # fill the parent frame
my_text.insert(1.0 , "There is a problem that use to test if the text widge is fixed. If the line goes to the next line, it will be correct.")

my_button = Button(root , text="Increase Font" , width=13 , font="arial 11" , command=increase_font)
my_button.pack(pady=5)

mainloop()

In [ ]:

import tkinter as tk

root = tk.Tk()

b1 = tk.Button(root, text='b1').pack(side=tk.LEFT)
b2 = tk.Button(root, text='b2').pack(side=tk.LEFT)
b3 = tk.Button(root, text='b3').pack(side=tk.LEFT)
b4 = tk.Button(root, text='b4').pack(side=tk.RIGHT)

root.mainloop()

In [ ]:
import os
import json

def read_new_words(dir):
    """ Read new words booklet, if it doesn't exist, create one.
    """
    if not os.path.isfile(dir):
        open(dir, 'a').close()

    with open(dir, 'r') as file:
        try:
            content = json.loads(file.read())
        except json.JSONDecodeError:
            content = {}

    return content

W_DIR = 'data/new words.json'
new_ws = read_new_words(W_DIR)

In [ ]:
new_dict = dict(sorted(new_ws.items(), key=lambda item: item[1][1], reverse=True))
for i in new_dict.items():
    print(f'{i[0]}: {i[1][0]}')

In [ ]:
from tkinter import *

ws = Tk()
ws.title('Learning Cantonese')
ws.geometry('600x400')

var = StringVar()

l = Label(ws, width=4, textvariable=var)
l.pack(fill=X)

list_items = {"Arushi": 22, "Mani": 21, "Haritha": 21}
  
lb = Listbox(ws, selectmode = "single")
for item in list_items.items():
	lb.insert('end', f'{item[0]}: {item[1]}')
      
lb.pack()

def delete_selection(lb):
    global var
    value = lb.curselection()

    lb.delete(value[0])
    if len(value) > 0:
        list_items.pop(list(list_items)[value[0]])

def select_item(lb):
    if lb.curselection() == ():
        var.set('no selection')
    else:
        value = lb.curselection()
        var.set(lb.get(value))

b = Button(ws, text='Delete', command=lambda: delete_selection(lb))
b.pack()

b2 = Button(ws, text='Add selection', command=lambda: select_item(lb))
b2.pack()

ws.mainloop()


In [ ]:
from PIL import ImageTk, Image


In [ ]:
import tkinter as tk

root = tk.Tk()
root.title('Pack Demo')
root.geometry("350x200")

# box 1
box3 = tk.Label(root, text="Box 3", bg="green", fg="white")
box3.pack(ipadx=20, ipady=20, fill=tk.BOTH, expand=1, anchor=tk.W)


# box 1
box1 = tk.Label(root, text="Box 1", bg="green", fg="white")
box1.pack(ipadx=20, ipady=20, fill=tk.X, expand=1, anchor=tk.W, side=tk.LEFT)

# box 2
box2 = tk.Label(root, text="Box 2", bg="red", fg="white")
box2.pack(ipadx=20, ipady=20, expand=1, anchor=tk.W, side=tk.RIGHT)

# box 1
box4 = tk.Label(root, text="Box 4", bg="green", fg="white")
box4.pack(ipadx=20, ipady=20, fill=tk.BOTH, expand=1, anchor=tk.W)

root.mainloop()

In [ ]:
import tkinter as tk
from tkinter import ttk

root = tk.Tk()
root.title('Pack Demo')
root.geometry("300x200")

ipadding = {'ipadx': 10, 'ipady': 10}

# place widgets top down
label1 = tk.Label(root, text='Box 1', bg="red", fg="white")
label1.pack(fill=tk.X)

label2 = tk.Label(root, text='Box 2', bg="green", fg="white")
label2.pack(fill=tk.X)

# place widgets side by side
label4 = tk.Label(root, text='Left', bg="cyan", fg="black")
label4.pack(expand=True, fill=tk.BOTH, side=tk.LEFT)

label5 = tk.Label(root, text='Center', bg="magenta", fg="black")
label5.pack(expand=True, fill=tk.BOTH, side=tk.LEFT)

label6 = tk.Label(root, text='Right', bg="yellow", fg="black")
label6.pack(expand=True, fill=tk.BOTH, side=tk.LEFT)

label3 = tk.Label(root, text='Box 3', bg="blue", fg="white")
label3.pack(fill=tk.X)

root.mainloop()

#### Stimulate pages

In [5]:
# ------------------------------------------------------------------
#  Select Song Page
#  -- Create a listbox to show all songs
#  -- Create buttons for next step
# ------------------------------------------------------------------
from tkinter import *
import learnCantonese as lc
import glob
import utils as u
import pandas as pd

import requests
from bs4 import BeautifulSoup

def get_song(lb):
    global SONG

    if lb.curselection() != ():
        value = lb.get(lb.curselection())
        SONG.set(value)

def songPage_2_mainPage(ws1):
    pass

def search_song(entry, data, lb):

    song_df = pd.read_csv('data/songs info.csv').drop_duplicates(subset='song id').drop_duplicates(subset=['singer', 'song'])
    all_res = song_df[song_df['song'] == entry.get()]

    Update(all_res[['song id', 'singer', 'song']].values.tolist(), lb)
    entry.delete(0, END)

def search_res(event, lb):
    global SONG_LIST

    val = event.widget.get()

    if val == '':
        data = SONG_LIST
    else:
        data = [item for item in SONG_LIST if val.lower() in item.lower()]
    Update(data, lb)

def Update(data, lb):
	
	lb.delete(0, 'end')

	# put new data
	for item in data:
		lb.insert('end', item)

def download_lyric(song_info):

    lyric_f = open(f'data/songs/{song_info[2]}.txt', 'w')
    url = f'https://www.feitsui.com/zh-hans/lyrics/{song_info[0]}'
    parent = get_part(url, 'body')

    for p in parent.find('article').find_all('p'):
        for l in p.contents:
            if '<' not in str(l):
                lyric_f.write(str(l).replace('翡翠粤语歌词', '').strip() + '\n')
    lyric_f.close()

def download_event(lb):
    global SONG_LIST

    # Get song id
    if lb.curselection() != ():
        song_info = lb.get(lb.curselection())
    
    download_lyric(song_info)
    Update(SONG_LIST, lb)
    

# Song Page
ws1 = Tk()
ws1.title('Select Songs')
u.center_window(ws1)

SONG = StringVar()

# Create components
searchframe = Frame(ws1)
listframe = Frame(ws1)
buttonframe = Frame(ws1)

# search frame
b = Button(searchframe, text='Back', command=lambda: songPage_2_mainPage(ws1), width=10, bg='#616161', fg='white')

entry_default = StringVar()
entry_default.set('Search Songs')
search_box = Entry(searchframe, textvariable=entry_default)

ser_b = Button(searchframe, text='Search Online', command=lambda: search_song(search_box, SONG_LIST, lb), width=10, bg='#616161', fg='white')

def on_click(event):
    """ Clear search box when click on it
    """

    event.widget.delete(0, END)

search_box.bind("<Button-1>", on_click)

# song list frame
all_f = glob.glob('data/songs/*.txt')
SONG_LIST = [x[11:-4] for x in all_f]

lb = Listbox(listframe, listvariable=SONG_LIST, borderwidth=0, bg='#3D3D3D', selectbackground='#2B57B7', activestyle='none', selectmode=SINGLE)
for s in SONG_LIST:
    lb.insert('end', s)
Update(SONG_LIST, lb)

# button frame
dl_b = Button(buttonframe, text='Download Lyric', command=lambda: download_event(lb), width=10, bg='#616161', fg='white')

searchframe.pack(side=TOP, fill=X)
b.pack(side=LEFT, padx=10)
search_box.pack(side=LEFT, fill=X)
search_box.bind('<KeyRelease>', lambda event: search_res(event, lb))
ser_b.pack(side=RIGHT, padx=10)

listframe.pack(fill='both', expand=1)
lb.pack(fill='both', expand=1, padx=10, pady=20)

buttonframe.pack()
dl_b.pack(side=LEFT, padx=10, pady=5)

ws1.mainloop()

In [8]:
# Highlight new words
from tkinter import *
import learnCantonese as lc
W_DIR = 'data/new words copy.json'

new_w = dict(lc.read_new_words(W_DIR).items())
new_w_list = [f'{i[0]}: {i[1][0]}' for i in new_w.items()]

root = Tk()
root.geometry('400x280')

selection = StringVar()

label = Label(root, textvariable=selection)
label.pack(fill=X)

listbox = Listbox(root)
listbox.pack(expand=1, fill="both")

# inserting some items
listbox.insert("end", "A list item")

for item in new_w.items():
    listbox.insert("end", f'{item[0]}: {item[1][0]}')
    if item[1][-1] == 1:
        listbox.itemconfig("end", {'fg': 'light green'})

def add_selection():
    value = listbox.get(listbox.curselection())
    selection.set(value)

def highlight_selection(new_w):
    value = listbox.get(listbox.curselection())
    selection.set(value)
    
    if new_w[value.split(':')[0]][-1] == 0:
        listbox.itemconfig(listbox.curselection()[0], {'fg': 'light green'})
        new_w[value.split(':')[0]][-1] = 1
    else:
        listbox.itemconfig(listbox.curselection()[0], {'fg': 'white'})
        new_w[value.split(':')[0]][-1] = 0

b = Button(root, text='Select', command=add_selection)
b.pack(side=LEFT)

b2 = Button(root, text='highlight', command=lambda: highlight_selection(new_w))
b2.pack(side=LEFT)

root.mainloop()

lc.output_new_words(W_DIR, new_w)

In [ ]:
import re

def next_lyrics(display_c_l, cn_l, t, e):
    global N

    N += 1
    value = cn_l[N]
    display_c_l.set(value)

    t.delete("1.0", END)
    e.delete(0, END)
    e.focus_set()

def previous_lyrics(display_c_l, cn_l, t, e):
    global N

    N -= 1
    value = cn_l[N]
    display_c_l.set(value)

    t.delete("1.0", END)
    e.delete(0, END)
    e.focus_set()

def check_lyrics(input_l, cn_l, en_l, w_dir, t, t2):
    global N

    if (len(input_l) == 0) and (t.get('1.0', END) == '\n'):
        t.insert(END, 'There is no input')

    elif t.get('1.0', END) in ['There is no input\n', '\n']:
        t.delete('1.0', END)

        cn_w = list(cn_l.replace(' ', ''))
        en_l = re.sub('\s+',' ', en_l).lstrip().split(' ')
        check_words = re.sub('\s+',' ', input_l).lstrip().split(' ')
        new_w = lc.read_new_words(w_dir)

        for w in range(len(check_words)):
            if (check_words[w] != en_l[w]) and ('/' not in en_l[w] or check_words[w] not in en_l[w].split('/')):
                t.insert(END, en_l[w], 'warning')
                t.insert(END, '  ')

                # add to word book
                lc.add_new_words(new_w, cn_w[w], en_l[w])
                lc.output_new_words(w_dir, new_w)

                if cn_w[w] not in t2.get("1.0", "end"):
                    t2.insert(END, f'{cn_w[w]}: ')
                    t2.insert(END, f'{en_l[w]}\t', 'warning')

            else:
                t.insert(END, f'{en_l[w]}  ')

# Fix duplicate check
from tkinter import *
import learnCantonese as lc

root = Tk()
root.geometry('400x280')

# Get lyrics
lyric_dir = 'data/songs/南昌街王子.txt'
clean_l = lc.read_lyrics(lyric_dir)
cn_l, en_l = clean_l[::2], lc.split_words_tones(clean_l[1::2])
N = 0

display_c_l = StringVar()
display_c_l.set(cn_l[N])

# components to input cantonese
frame = Frame(root)

l = Label(frame, textvariable=display_c_l, bg='#323232', font=("arial", 14), height=2)

e = Entry(frame, bg='black')
e.focus_set()

def clear_entry():
    e.delete(0, 'end')

clear_b = Button(frame, text='x', command=lambda:clear_entry(), width=20, height=20, bg='#616161', fg='white')

# textframe = Frame(root)
t = Text(frame, height=2, font=("arial", 14))
t.tag_config('warning', foreground='#A6FF2E')

# buttons
buttonframe = Frame(root)
b1 = Button(buttonframe, text='Check', command=lambda: check_lyrics(e.get(), cn_l[N], en_l[N], W_DIR, t, t2), bg='#616161', fg='white', width=200)
b2 = Button(buttonframe, text='->', command=lambda: next_lyrics(display_c_l, cn_l, t, e), bg='#616161', fg='white', width=100)
b3 = Button(buttonframe, text='<-', command=lambda: previous_lyrics(display_c_l, cn_l, t, e), bg='#616161', fg='white', width=100)

# words text
frame2 = Frame(root)
t2 = Text(frame2, font=("Helvetica", 14))
t2.tag_config('warning', foreground='#A6FF2E')

# show components on screen
frame.pack()

l.pack(fill='x', padx=10, pady=5)
e.pack(fill='x', padx=10, pady=5)
# clear_b.pack(padx=10, pady=5)
t.pack(fill='x', expand=True, padx=10, pady=5)

buttonframe.pack()
b3.pack(padx=10, pady=5, side=LEFT)
b1.pack(padx=10, pady=5, side=LEFT)
b2.pack(padx=10, pady=5, side=LEFT)

frame2.pack()
t2.pack(fill='x', padx=10, pady=5)

root.bind('<Return>', lambda event: check_lyrics(e.get(), cn_l[N], en_l[N], W_DIR, t, t2))
root.bind('<Right>', lambda event: next_lyrics(display_c_l, cn_l, t, e))
root.bind('<Left>', lambda event: previous_lyrics(display_c_l, cn_l, t, e))

root.mainloop()

In [2]:
int(...)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'ellipsis'

In [3]:
import tkinter as tk

root = tk.Tk()

listbox = tk.Listbox(root)
scrollbar = tk.Scrollbar(root)

listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=listbox.yview)

listbox.pack(side='left')
scrollbar.pack(side='right', fill='y')

for i in range(100):
    listbox.insert('end', 'Item %d' % i)

root.mainloop()

: 

In [2]:
import glob

all_f = glob.glob('data/songs/*.txt')
SONG_LIST = [x[11:-4] for x in all_f]
SONG_LIST

['奇洛李维斯回信',
 '岁月如歌',
 '再见二丁目',
 '高山低谷',
 '南昌街王子',
 '爱与诚',
 '钟无艳',
 '南瓜车',
 '爱得太迟',
 '富士山下',
 '甜蜜蜜',
 '现代爱情故事',
 '十七岁',
 '我们都是这样长大的',
 '恋爱大过天',
 '红绿灯',
 '寻找独角兽',
 '一丝不挂',
 '有只雀仔',
 '黄色大门',
 '一人有一个梦想',
 '祝福',
 '你们的幸福',
 '下一站天后',
 '饮歌',
 '年度之歌',
 '笑看风云',
 '心淡',
 '慕容雪',
 '简爱',
 '黑洞',
 '喜帖街',
 '灰姑娘',
 '我中意',
 '男孩像你',
 '孤单的心痛',
 '谁愿放手',
 '勇',
 '分分钟需要你',
 '死性不改']

In [3]:
from pinyin import get

sorted(SONG_LIST, key=get)

['富士山下',
 '分分钟需要你',
 '高山低谷',
 '孤单的心痛',
 '黄色大门',
 '红绿灯',
 '黑洞',
 '灰姑娘',
 '简爱',
 '恋爱大过天',
 '慕容雪',
 '年度之歌',
 '南昌街王子',
 '南瓜车',
 '男孩像你',
 '你们的幸福',
 '奇洛李维斯回信',
 '十七岁',
 '谁愿放手',
 '岁月如歌',
 '死性不改',
 '甜蜜蜜',
 '我们都是这样长大的',
 '我中意',
 '现代爱情故事',
 '笑看风云',
 '下一站天后',
 '寻找独角兽',
 '心淡',
 '喜帖街',
 '一人有一个梦想',
 '一丝不挂',
 '饮歌',
 '勇',
 '有只雀仔',
 '祝福',
 '钟无艳',
 '再见二丁目',
 '爱得太迟',
 '爱与诚']